A copy from https://github.com/Heumi/BEGAN-tensorflow/tree/master/src/function

To test the difference

http://www.foldl.me/uploads/2015/conditional-gans-face-generation/paper.pdf

https://github.com/nightrome/really-awesome-gan

https://arxiv.org/pdf/1705.09966.pdf

https://arxiv.org/pdf/1703.05192.pdf

https://arxiv.org/pdf/1606.03657.pdf

https://arxiv.org/pdf/1704.02166.pdf

https://arxiv.org/pdf/1709.03842.pdf

In [1]:
import os
import scipy.misc as scm
import numpy as np

def make_project_dir(project_dir):
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)
        os.makedirs(os.path.join(project_dir, 'models'))
        os.makedirs(os.path.join(project_dir, 'result'))
        os.makedirs(os.path.join(project_dir, 'result_test'))


def get_image(img_path):
    img = scm.imread(img_path)/255. - 0.5
    img = img[..., ::-1]  # rgb to bgr
    return img

def get_label(path, size):
    label = int(path[-5])
    one_hot = np.zeros(size)
    one_hot[ label ] = 1.
    return one_hot

def inverse_image(img):
    img = (img + 0.5) * 255.
    img[img > 255] = 255
    img[img < 0] = 0
    img = img[..., ::-1] # bgr to rgb
    return img

def pair_expressions(paths):
    subject_exprs = []
    subject_pairs = []
    all_pairs = []
    last_subject = 0

    # Pair all expression of a subject
    for path in paths:
        subject = int(path[-9:-6])

        if subject != last_subject and last_subject != 0:
            subject_pairs = [(x, y) for x in subject_exprs for y in subject_exprs]
            all_pairs.extend(subject_pairs)
            subject_exprs = []

        subject_exprs.append(path)
        last_subject = subject

    # Last subject
    subject_pairs = [(x, y) for x in subject_exprs for y in subject_exprs]
    all_pairs.extend(subject_pairs)
    return all_pairs

def get_shape_c(tensor): # static shape
    return tensor.get_shape().as_list()


In [2]:
import tensorflow as tf
import numpy as np


def conv2d(x, filter_shape, bias=True, stride=1, padding="SAME", name="conv2d"):
    kw, kh, nin, nout = filter_shape
    pad_size = (kw - 1) / 2

    if padding == "VALID":
        x = tf.pad(x, [[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], "SYMMETRIC")

    initializer = tf.random_normal_initializer(0., 0.02)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=filter_shape, initializer=initializer)
        x = tf.nn.conv2d(x, weight, [1, stride, stride, 1], padding=padding)

        if bias:
            b = tf.get_variable("bias", shape=filter_shape[-1], initializer=tf.constant_initializer(0.))
            x = tf.nn.bias_add(x, b)
    return x


def fc(x, output_shape, bias=True, name='fc'):
    shape = x.get_shape().as_list()
    dim = np.prod(shape[1:])
    x = tf.reshape(x, [-1, dim])
    input_shape = dim

    initializer = tf.random_normal_initializer(0., 0.02)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=[input_shape, output_shape], initializer=initializer)
        x = tf.matmul(x, weight)

        if bias:
            b = tf.get_variable("bias", shape=[output_shape], initializer=tf.constant_initializer(0.))
            x = tf.nn.bias_add(x, b)
    return x


def pool(x, r=2, s=1):
    return tf.nn.avg_pool(x, ksize=[1, r, r, 1], strides=[1, s, s, 1], padding="SAME")


def l1_loss(x, y):
    return tf.reduce_mean(tf.abs(x - y))


def resize_nn(x, size):
    return tf.image.resize_nearest_neighbor(x, size=(int(size), int(size)))

In [3]:
import glob
import time
import numpy as np
import tensorflow as tf

class op_base:
    def __init__(self, sess, project_name):
        self.sess = sess

        # Train
        self.flag = True #args.flag
        self.gpu_number = 0 #args.gpu_number
        self.project = project_name #"test_began" #args.project

        # Train Data
        self.data_dir = "./Face_data/Faces_with_expression_label/dataset_64x64" #args.data_dir #./Data
        self.dataset = "expr" #args.dataset  # celeba
        self.data_size = 64 #args.data_size  # 64 or 128
        self.data_opt = "crop" #args.data_opt  # raw or crop
        self.data_label_vector_size = 7 #size of one-hot-encoded label vector

        # Train Iteration
        self.niter = 500 #50 #args.niter
        self.niter_snapshot = 200 #2440 #args.nsnapshot
        self.max_to_keep = 5 #args.max_to_keep

        # Train Parameter
        self.batch_size = 32*6 #16 #args.batch_size
        self.learning_rate =  5e-5 # <----WGAN #1e-4  <-- BEGAN #args.learning_rate
        self.mm = 0.5 #args.momentum
        self.mm2 = 0.999 #args.momentum2
        self.lamda = 0.001 #args.lamda
        self.gamma = 2.0 #args.gamma
        self.filter_number = 64 #args.filter_number
        self.input_size = 64 #args.input_size
        self.embedding = 64 #args.embedding

        # Result Dir & File
        self.project_dir = 'assets/{0}_{1}_{2}_{3}/'.format(self.project, self.dataset, self.data_opt, self.data_size)
        self.ckpt_dir = os.path.join(self.project_dir, 'models')
        self.model_name = "{0}.model".format(self.project)
        self.ckpt_model_name = os.path.join(self.ckpt_dir, self.model_name)

        # etc.
        if not os.path.exists('assets'):
            os.makedirs('assets')
        make_project_dir(self.project_dir)

    def load(self, sess, saver, ckpt_dir):
        ckpt = tf.train.get_checkpoint_state(ckpt_dir)
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(ckpt_dir, ckpt_name))

In [4]:
import glob
import time
import datetime


class Operator(op_base):
    def __init__(self, sess, project_name):
        op_base.__init__(self, sess, project_name)
        self.build_model()

    def build_model(self):
        # Input placeholder
        self.x = tf.placeholder(tf.float32, shape=[None, self.input_size, self.input_size, 3], name='x')
        self.y = tf.placeholder(tf.float32, shape=[None, self.data_size, self.data_size, 3], name='y')
        self.c = tf.placeholder(tf.float32, shape=[None, self.data_label_vector_size], name='c')
        self.lr = tf.placeholder(tf.float32, name='lr')

        # Generator
        self.recon_gen = self.generator_decoder(self.generator_encoder(self.x), self.c)

        # Discriminator (Critic)
        d_real = self.discriminator(self.y, self.c)
        d_fake = self.discriminator(self.recon_gen, self.c, reuse=True)

        # Loss --------------------------------------------------
        wd = tf.reduce_mean(d_real) - tf.reduce_mean(d_fake)
        self.d_loss = -wd
        self.g_loss = -tf.reduce_mean(d_fake)
        #--------------------------------------------------------
        
        
        # Variables
        g_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "gen_")
        d_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "disc_")

        
        # Optimizer -----------------------------------------------
        
        d_step_ = tf.train.RMSPropOptimizer(learning_rate=self.lr).minimize(self.d_loss, var_list=d_vars)
        with tf.control_dependencies([d_step_]):
            self.opt_d = tf.group(*(tf.assign(var, tf.clip_by_value(var, -0.01, 0.01)) for var in d_vars))
        self.opt_g = tf.train.RMSPropOptimizer(learning_rate=self.lr).minimize(self.g_loss, var_list=g_vars)
        #-----------------------------------------------------------


        # initializer
        self.sess.run(tf.global_variables_initializer())

        # tf saver
        self.saver = tf.train.Saver(max_to_keep=(self.max_to_keep))

        try:
            self.load(self.sess, self.saver, self.ckpt_dir)
        except:
            # save full graph
            self.saver.save(self.sess, self.ckpt_model_name, write_meta_graph=True)

        # Summary
        if self.flag:
            tf.summary.scalar('loss/loss', self.d_loss + self.g_loss)
            tf.summary.scalar('loss/g_loss', self.g_loss)
            tf.summary.scalar('loss/d_loss', self.d_loss)
            tf.summary.scalar('misc/WassDist', wd)
            self.merged = tf.summary.merge_all()
            self.writer = tf.summary.FileWriter(self.project_dir, self.sess.graph)

    def train(self, train_flag):
        # load data
        data_path = self.data_dir

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            data = pair_expressions(data)
            np.save(data_path + '.npy', data)

        # initial parameter
        start_time = time.time()
        lr = np.float32(self.learning_rate)
        self.count = 0
        
        n_critic = 5
        small_batch_size = int(self.batch_size / (n_critic + 1))

        for epoch in range(self.niter):
            random_order = np.random.permutation(len(data))
            data = [data[i] for i in random_order[:]]
            
            batch_idxs = len(data) // self.batch_size

            for idx in range(0, batch_idxs):
                self.count += 1

                batch_files = data[idx * self.batch_size: (idx + 1) * self.batch_size]
                batch_inputs = [get_image(batch_file[0]) for batch_file in batch_files]
                batch_targets = [get_image(batch_file[1]) for batch_file in batch_files]
                batch_target_labels = [get_label(batch_file[1], self.data_label_vector_size) for batch_file in batch_files]

                # train D
                for i in range(n_critic):
                    one_batch_in = batch_inputs[i*small_batch_size : (i+1)*small_batch_size]
                    one_batch_target = batch_targets[i*small_batch_size : (i+1)*small_batch_size]
                    one_batch_target_labels = batch_target_labels[i*small_batch_size : (i+1)*small_batch_size]
                    
                    d_opt = [self.opt_d, self.d_loss, self.merged]
                    feed_dict_d = {self.x: one_batch_in, self.y: one_batch_target, self.c: one_batch_target_labels,
                                 self.lr: lr}
                    
                    _, loss_d, summary = self.sess.run(d_opt, feed_dict=feed_dict_d)
                    
                    
                # train G
                one_batch_in = batch_inputs[n_critic*small_batch_size : (n_critic+1)*small_batch_size]
                one_batch_target_labels = batch_target_labels[n_critic*small_batch_size : (n_critic+1)*small_batch_size]
                
                g_opt = [self.opt_g, self.g_loss]
                feed_dict_g = {self.x: one_batch_in, self.c: one_batch_target_labels, self.lr: lr}
                _, loss_g = self.sess.run(g_opt, feed_dict=feed_dict_g)
               

                if self.count % 5 == 1:
                    print("Epoch: [%2d] [%4d/%4d] time: %4.4f, "
                          "loss_g: %.4f, loss_d: %.4f"
                          % (epoch, idx, batch_idxs, time.time() - start_time,
                             loss_g, loss_d))

                # write train summary
                self.writer.add_summary(summary, self.count)

                # Test during Training
                if self.count % self.niter_snapshot == (self.niter_snapshot - 1):                    
                    # save & test
                    self.saver.save(self.sess, self.ckpt_model_name, global_step=self.count, write_meta_graph=False)
                    self.test(train_flag)

    def test(self, train_flag=True):
        # generate output
        img_num =  36 #self.batch_size
        display_img_num = int(img_num / 3)
        img_size = self.data_size

        output_f = int(np.sqrt(img_num))
        im_output_gen = np.zeros([img_size * output_f, img_size * output_f, 3])
        
        # load data
        data_path = self.data_dir

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            data = pair_expressions(data)
            np.save(data_path + '.npy', data)

        print('Test data shuffle ....')
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]
        print('Test data shuffle Done')
        
        batch_files = data[0: display_img_num]
        test_inputs = [get_image(batch_file[0]) for batch_file in batch_files]
        test_inputs_o = [scm.imread((batch_file[0])) for batch_file in batch_files]
        test_targets = [scm.imread((batch_file[1])) for batch_file in batch_files]
        test_target_labels = [get_label(batch_file[1], self.data_label_vector_size) for batch_file in batch_files]

        output_gen = (self.sess.run(self.recon_gen, feed_dict={self.x: test_inputs, 
                                                               self.c: test_target_labels}))  # generator output

        output_gen = [inverse_image(output_gen[i]) for i in range(display_img_num)]

        for i in range(output_f): # row
            for j in range(output_f): # col
                if j % 3 == 0: # input img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_inputs_o[int(j / 3) + (i * int(output_f / 3))]
                elif j % 3 == 1: # output img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = output_gen[int(j / 3) + (i * int(output_f / 3))]
                else: # target img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_targets[int(j / 3) + (i * int(output_f / 3))]

        labels = np.argmax(test_target_labels, axis=1)
        label_string = ''.join(str(int(l)) for l in labels)
        # output save
        if train_flag:
            scm.imsave(self.project_dir + '/result/' + str(self.count) + '_' + label_string 
                       + '_output.bmp', im_output_gen)
        else:
            now = datetime.datetime.now()
            nowDatetime = now.strftime('%Y-%m-%d_%H:%M:%S')
            scm.imsave(self.project_dir + '/result_test/gen_{}_'.format(nowDatetime) 
                       + label_string + '_output.bmp', im_output_gen)


In [5]:

class CWGAN(Operator):
    def __init__(self, sess, project_name):
        Operator.__init__(self, sess, project_name)
        
    def generator_encoder(self, x, reuse=None):
        with tf.variable_scope('gen_') as scope:
            if reuse:
                scope.reuse_variables()

            f = self.filter_number
            h = self.embedding
            p = "SAME"

            x = conv2d(x, [3, 3, 3, f], stride=1,  padding=p,name='conv1_enc_a')
            x = tf.nn.elu(x)

            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv2_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv2_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, f, 2 * f], stride=1,  padding=p,name='conv3_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 2 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 3 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = conv2d(x, [1, 1, 4 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_0')
                x = pool(x, r=2, s=2)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_b')
                x = tf.nn.elu(x)

            x = fc(x, h, name='enc_fc')
        return x

    def generator_decoder(self, x, c, reuse=None):
        with tf.variable_scope('gen_') as scope:
            if reuse:
                scope.reuse_variables()

            w = self.data_size
            f = self.filter_number
            p = "SAME"
            
            x = tf.concat([x, c], axis=1) # adding label information

            x = fc(x, 8 * 8 * f, name='fc')
            x = tf.reshape(x, [-1, 8, 8, f])

            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv1_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv1_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = resize_nn(x, w / 8)
                x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv2_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv2_b')
                x = tf.nn.elu(x)

            x = resize_nn(x, w / 4)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv3_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv3_b')
            x = tf.nn.elu(x)

            x = resize_nn(x, w / 2)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv4_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p, name='conv4_b')
            x = tf.nn.elu(x)

            x = resize_nn(x, w)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv5_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv5_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [3, 3, f, 3], stride=1,  padding=p,name='conv6_a')
        return x

    def discriminator(self, x, c, reuse=None):
        with tf.variable_scope('disc_') as scope:
            if reuse:
                scope.reuse_variables()

            f = self.filter_number
            h = self.embedding
            p = "SAME"
            c_len = self.data_label_vector_size

            x = conv2d(x, [3, 3, 3, f], stride=1,  padding=p,name='conv1_enc_a')
            x = tf.nn.elu(x)
            
            # Add y as a channel to dis1 layer as described in IcGAN paper
            x = tf.concat([x, tf.tile(tf.reshape(c, [-1, 1, 1, get_shape_c(c)[-1]]),\
                                     [1, x.get_shape().as_list()[1], x.get_shape().as_list()[2], 1])],\
                              axis=3)

            x = conv2d(x, [3, 3, f + c_len, f], stride=1,  padding=p,name='conv2_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv2_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, f, 2 * f], stride=1,  padding=p,name='conv3_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 2 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 3 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = conv2d(x, [1, 1, 4 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_0')
                x = pool(x, r=2, s=2)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_b')
                x = tf.nn.elu(x)

            x = fc(x, h, name='enc_fc')
            x = tf.nn.sigmoid(x)
        return x


In [6]:
import distutils.util
import os
import tensorflow as tf

''' config settings '''

project_name = "CWGAN_exprs_1"
train_flag = True

'''-----------------'''

gpu_number = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #args.gpu_number

with tf.device('/gpu:{0}'.format(gpu_number)):
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.90)
    config = tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)

    with tf.Session(config=config) as sess:
        model = CWGAN(sess, project_name)

        # TRAIN / TEST
        if train_flag:
            model.train(train_flag)
        else:
            model.test(train_flag)

Epoch: [ 0] [   0/  52] time: 1.6994, loss_g: -0.5000, loss_d: -0.0000
Epoch: [ 0] [   5/  52] time: 5.2790, loss_g: -0.5000, loss_d: -0.0000
Epoch: [ 0] [  10/  52] time: 8.8535, loss_g: -0.5000, loss_d: -0.0000
Epoch: [ 0] [  15/  52] time: 12.4328, loss_g: -0.5000, loss_d: -0.0000
Epoch: [ 0] [  20/  52] time: 16.0117, loss_g: -0.5000, loss_d: -0.0000
Epoch: [ 0] [  25/  52] time: 19.5968, loss_g: -0.5000, loss_d: -0.0000
Epoch: [ 0] [  30/  52] time: 23.1878, loss_g: -0.5000, loss_d: -0.0000
Epoch: [ 0] [  35/  52] time: 26.7818, loss_g: -0.5000, loss_d: -0.0000


KeyboardInterrupt: 